# ETL Process

In [1]:
#Dependencies - Modules
import os
import csv

#Dependencies
import pandas as pd
import numpy as np



In [2]:
#Simple csv reading
'''
csvpath = os.path.join('.', 'Resources', 'ORG_OTC_FCT_DET_reduced.csv')
with open(csvpath, newline='') as csvfile:

    # CSV reader specifies delimiter and variable that holds contents
    csvreader = csv.reader(csvfile, delimiter=',')

    print(csvreader)

    # Read the header row first (skip this step if there is now header)
    csv_header = next(csvreader)
    print(f"CSV Header: {csv_header}")

    # Read each row of data after the header
    for row in csvreader:
        print(row)
'''

'\ncsvpath = os.path.join(\'.\', \'Resources\', \'ORG_OTC_FCT_DET_reduced.csv\')\nwith open(csvpath, newline=\'\') as csvfile:\n\n    # CSV reader specifies delimiter and variable that holds contents\n    csvreader = csv.reader(csvfile, delimiter=\',\')\n\n    print(csvreader)\n\n    # Read the header row first (skip this step if there is now header)\n    csv_header = next(csvreader)\n    print(f"CSV Header: {csv_header}")\n\n    # Read each row of data after the header\n    for row in csvreader:\n        print(row)\n'

In [3]:
#Using pandas to create df
#loading csv - the correct encoding must be used to read the CSV in pandas
file_to_load = "Resources/ORG_OTC_FCT_DET_reduced.csv"
original_organ_transp_data = pd.read_csv(file_to_load,encoding="ISO-8859-1")
#print(original_organ_transp_data.head)



In [4]:
#Renamed columns to load csv into MySQL - need to use '_' in between spaces
original_organ_transp_data.columns = original_organ_transp_data.columns.str.strip().str.lower().str.replace(' ', '_')
#original_organ_transp_data


In [5]:
#store csv to lead into mysql
#original_organ_transp_data.to_csv('resources/organ_data.csv')


# MySQL Process

In [5]:
# SQL Alchemy
from sqlalchemy import create_engine

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()



In [6]:
# Create Engine and Pass in MySQL Connection - exercise 10-1-10 Creating Sqlite 
engine = create_engine('sqlite:///db/organ_transplant.sqlite')
conn = engine.connect()


In [7]:
import os

import pandas as pd
#import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template, request, redirect
from flask_sqlalchemy import SQLAlchemy

#################################################
# Flask Setup
#################################################
app = Flask(
    __name__, 
    template_folder='templates/light-bootstrap-dashboard-master',
    static_url_path='/assets',
    static_folder='templates/light-bootstrap-dashboard-master/static'
)

#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///organ_transplant.db"
# Create DB object to pass Flask app to it - SQLAlchemy object that can be used in Flask
db = SQLAlchemy(app)

#Create database model (emoji exercise 15.3.3)
#Create a class that defines what the table is and what columns are on the table
class Organ_data(db.Model):
    __tablename__ = 'organ_data'
    #Define the Columns
    id = db.Column('id', db.Integer, primary_key=True) #takes in 3 arguments SQLAlchemy requires a primary key - https://www.youtube.com/watch?v=Tu4vRU4lt6k
    organ_transplantation_type = db.Column(db.String(64))
    organ_procurement_organization_name = db.Column(db.String(64))
    organ_procurement_organization_city = db.Column(db.String(64))
    county = db.Column(db.String(64))
    state = db.Column(db.String(64))
    state_abbreviation = db.Column(db.String(64))
    data_warehouse_record_create_date_text = db.Column(db.String(64))
    geocoding__primary_x_coordinate = db.Column(db.Float)
    geocoding_primary_y_coordinate = db.Column(db.Float)
   
    def __repr__(self):
        return '<Organ_data %r>' % (self.name)



/anaconda3/envs/PythonData/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:835: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [13]:
# Create database tables
@app.before_first_request
def setup():
    # Recreate database each time for demo
    # db.drop_all()
    db.create_all()
    
# Query test for top 10 data points
@app.route("/organ_transplantation_type")
def top_organs_data():
   # Query: this will return the top 10 organs based on most transplants done
    results = db.session.query(Organ.organ_transplantation_type).\
        order_by(Organ.organ_transplantation_type.desc()).\
        limit(10).all
    df = pd.DataFrame(results, columns=['organ'])

if __name__ == '__main__':
    app.run(debug=True)


AssertionError: View function mapping is overwriting an existing endpoint function: top_organs_data

In [ ]:
#Create route that renders dashboard.html template
@app.route("/")
def home():
    active_tab = "dashboard"
    return render_template("dashboard.html", active = active_tab)

@app.route("/etl")
def etl():
    active_tab = "etl"
    return render_template("user.html", active = active_tab )

@app.route("/table")
def table():
    active_tab = "table"
    return render_template("table.html", active = active_tab )

@app.route("/map")
def map():
    active_tab = "maps"
    return render_template("maps.html",  active = active_tab)

@app.route("/funfacts")
def funfacts():
    active_tab = "funfacts"
    return render_template("notifications.html",  active = active_tab)

@app.route("/chartdata", methods=['POST'])
def chartdata():
    print("pepee")


# Query test for top 10 data points
@app.route("/organ_transplantation_type")
def top_organs_data():
    
   # Query: this will return the top 10 organs based on most transplants done
    results = db.session.query(Organ.organ_transplantation_type).\
        order_by(Organ.organ_transplantation_type.desc()).\
        limit(10).all
#   results = db.session.query(Organ_data.organ_transplantation_type, Organ_data.count(Organ_data.type)).group_by(Organ_data.type).limit(5).all()
    
    organ_type = [result[0] for result in results]
    id = [result[1] for result in results]

    trace = {
        "x": organ_type,
        "y": id,
        "type": "bar"
    }

    return jsonify(trace)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
